In [1]:
# import os
# os.environ["R_HOME"] = "/home/ld/anaconda3/envs/r/lib/R"
import sys
sys.path.append('../')
import scDesign3

The R project used is located at D:/Code_env/R-4.3.1


In [2]:
import anndata as ad
example_sce = ad.read_h5ad('test_data/VISIUM.h5ad')
example_sce.obs['test'] = ['test' for i in range(example_sce.n_obs)]
example_sce = example_sce[0:10,0:10]

In [3]:
example_sce

View of AnnData object with n_obs × n_vars = 10 × 10
    obs: 'nCount_Spatial', 'nFeature_Spatial', 'nCount_SCT', 'nFeature_SCT', 'SCT_snn_res.0.8', 'seurat_clusters', 'spatial1', 'spatial2', 'cell_type', 'test'
    var: 'name'

construct_data interface

In [ ]:
## windows
bpparam = scDesign3.get_bpparam("SnowParam",stop_on_error = False)

## linux/mac
# bpparam = test.get_bpparam("MulticoreParam",stop_on_error = False)

In [4]:
# test = scDesign3.scDesign3(n_cores=3,parallelization="bpmapply",bpparam=bpparam)
test = scDesign3.scDesign3(n_cores=1,parallelization="mcmapply")

In [5]:
const_data = test.construct_data(
    anndata=example_sce,
    corr_formula="1",
    default_assay_name="counts",
    celltype='cell_type',
    spatial=['spatial1','spatial2'],
)

In [ ]:
const_data['dat']

In [ ]:
print(test.construct_data_res)

In [ ]:
# SummarizedExperiment = packages.importr('SummarizedExperiment')
# print(SummarizedExperiment.assay(test._sce,test._assay_use))
# print(SummarizedExperiment.colData(test._sce))
# print(SummarizedExperiment.rowData(test._sce))

fit_marginal interface

In [6]:
marginal = test.fit_marginal(
    # data=const_data,
    mu_formula="s(spatial1,spatial2,bs='gp',k=2)",
    sigma_formula="1",
    family_use="nb",
    usebam=False,
    trace=True,
)

In [ ]:
print(test.fit_marginal_res)

fit_copula interface

In [ ]:
copula = test.fit_copula(
    # family_use="nb",
    n_cores=1,
    # copula = "vine",
    parallelization="mcmapply",
    # parallelization="bpmapply",
    # bpparam=bpparam,
    important_feature="auto",
)

In [ ]:
print(copula)

extract paras interface

In [ ]:
paras = test.extract_para()

In [ ]:
print(paras)

simu_new interface


In [ ]:
new = test.simu_new()

In [ ]:
print(new)

scdesign3 interface

In [ ]:
whole_test = scDesign3.scDesign3(n_cores=1)
whole = whole_test.scdesign3(    
    anndata=example_sce,
    corr_formula="1",
    default_assay_name="counts",
    celltype='cell_type',
    spatial=['spatial1','spatial2'],
    mu_formula="s(spatial1,spatial2,bs='gp',k=2)",
    sigma_formula="1",
    family_use="nb",
    usebam=False,
    trace=True,
    important_feature="auto",
    # parallelization="mcmapply",
    # n_cores=1,
    )

In [ ]:
print(whole_test.whole_pipeline_res)

In [ ]:
whole["marginal_list"]

perform_lrt interface

In [8]:
from rpy2.rlike.container import OrdDict
from rpy2.robjects import NULL, ListVector, StrVector, conversion, default_converter, numpy2ri, pandas2ri, r

@default_converter.py2rpy.register(OrdDict)
def convert_ord_dict(ord_dict):
    with (default_converter + pandas2ri.converter + numpy2ri.converter).context():
        ord_dict = {k: conversion.get_conversion().py2rpy(v) for k, v in ord_dict.items()}
    return ListVector(ord_dict)


convert = numpy2ri.converter + default_converter + pandas2ri.converter

In [29]:
with convert.context():
    a = test.fit_marginal_res.rx2("Calb2")
    b = r("mgcv::gam(Sepal.Length~Sepal.Width,data=iris)")

print(b)

o{'coefficients': [ 6.52622255 -0.22336106], 'residuals': [-0.64445884 -0.95613937 -1.11146716 -1.23380326 -0.72212273 -0.25511441
 -1.16679494 -0.76679494 -1.47847547 -0.93380326 -0.29978662 -0.96679494
 -1.05613937 -1.55613937  0.16722169  0.15656612 -0.25511441 -0.64445884
  0.02254948 -0.57745052 -0.36679494 -0.59978662 -1.12212273 -0.68913105
 -0.96679494 -0.85613937 -0.76679494 -0.54445884 -0.56679494 -1.11146716
 -1.03380326 -0.36679494 -0.4104422  -0.08810609 -0.93380326 -0.81146716
 -0.24445884 -0.82212273 -1.45613937 -0.66679494 -0.74445884 -1.51249211
 -1.41146716 -0.74445884 -0.57745052 -1.05613937 -0.57745052 -1.21146716
 -0.39978662 -0.78913105  1.18853284  0.58853284  1.06619674 -0.51249211
  0.59918842 -0.20081158  0.51086895 -1.090156    0.72152453 -0.72314769
 -1.07950043  0.04386063 -0.03482822  0.22152453 -0.27847547  0.86619674
 -0.25613937 -0.12314769  0.16517178 -0.3678199   0.08853284  0.19918842
  0.3321801   0.19918842  0.52152453  0.74386063  0.89918842  0.84

In [30]:
a["fit"]

R[write to console]: Error in h(simpleError(msg, call)) : 
  在为'show'函数选择方法时评估'object'参数出了错: could not find function "fitfunc" 

d:\Code_env\Anaconda3\envs\scrpy\lib\site-packages\rpy2\robjects\robject.py:123: UserWarning: Invalid call to "show()" in R: Error in h(simpleError(msg, call)) : 
  在为'show'函数选择方法时评估'object'参数出了错: could not find function "fitfunc" 

  warnings.warn(f'Invalid call to "show()" in R: {rre}')
R[write to console]: Error in fitfunc(formula = mgcv_formula, family = "nb", data = dat_use,  : 
  could not find function "fitfunc"



RRuntimeError: Error in fitfunc(formula = mgcv_formula, family = "nb", data = dat_use,  : 
  could not find function "fitfunc"


In [26]:
print(test.fit_marginal_res.rx2("Calb2"))

$fit

Family: Negative Binomial(2.63) 
Link function: log 

Formula:
gene ~ s(spatial1, spatial2, bs = "gp", k = 2)

Estimated degrees of freedom:
2  total = 3 

REML score: 42.72744     

$warning
$warning[[1]]
$warning[[1]]$function_name
[1] "gam"

$warning[[1]]$type
[1] "warning"

$warning[[1]]$message
[1] "simpleWarning in Sys.setlocale(\"LC_CTYPE\", chloc): using locale code page other than 936 may cause problems\n"


$warning[[2]]
$warning[[2]]$function_name
[1] "gam"

$warning[[2]]$type
[1] "warning"

$warning[[2]]$message
[1] "simpleWarning in smooth.construct.gp.smooth.spec(object, dk$data, dk$knots): basis dimension reset to minimum possible\n"



$time
[1] 0.06491804         NA

$removed_cell
[1] NA


